In [ ]:
import simpletransformers
from simpletransformers.classification import ClassificationModel, ClassificationArgs
import pandas as pd
import logging
import wandb
from sklearn.metrics import accuracy_score

## merge messages

In [ ]:
df = pd.read_csv('aug_df.csv')

In [ ]:
df = df.sort_values(['conversation_id', 'message_id'])

In [ ]:
df = df.drop(df[df['conversation_id'] == 'SM2a210417aa6d1bf15995611740616f7b'].index)

In [ ]:
df = df.dropna()

In [ ]:
df['up'] = df['encoded_message'].shift(-1, fill_value = '')
df['down'] =  df['encoded_message'].shift(1, fill_value = '')
df['3_messages'] = df.down + ' ' + df.encoded_message + ' ' + df.up

In [ ]:
df['conversation_id'] = df['conversation_id'].astype('float64')
df['message_id'] = df['message_id'].astype('float64')

### merge: prior + current + next

In [ ]:
conv_list = df['conversation_id'].unique()
for i in conv_list:
    conv = df[df.conversation_id == i]
    first = min(conv.message_id)
    last = max(conv.message_id)
    
    ind_1 = df[df['message_id'] == first].index[0]
    df.loc[ind_1, '3_messages'] = df['encoded_message'][ind_1]+ ' ' + df['up'][ind_1]

    ind_last = df[df['message_id'] == last].index[0]
    df.loc[ind_last, '3_messages'] = df['down'][ind_last] + ' ' + df['encoded_message'][ind_last]


# prior + prior + current

In [ ]:
# df = df.dropna()

In [ ]:
df['down_down'] =  df['encoded_message'].shift(2, fill_value = '')

In [ ]:
df['2priors'] = df.down_down + ' ' + df.down + ' ' + df.encoded_message

In [ ]:
conv_list = df['conversation_id'].unique()
for i in conv_list:
    conv = df[df.conversation_id == i]
    first = min(conv.message_id)
    second = conv.nsmallest(2, 'message_id')[1:2].message_id.values[0]
    
    ind_1 = df[df['message_id'] == first].index[0]
    df.loc[ind_1, '2priors'] = df['encoded_message'][ind_1]

    ind_2 = df[df['message_id'] == second].index[0]
    df.loc[ind_2, '2priors'] = df['down'][ind_2] + ' ' + df['encoded_message'][ind_2]

# prior + current 

In [ ]:
df['1prior'] = df.down + ' ' + df.encoded_message

In [ ]:
conv_list = df['conversation_id'].unique()
for i in conv_list:
    conv = df[df.conversation_id == i]
    first = min(conv.message_id)
    
    ind_1 = df[df['message_id'] == first].index[0]
    df.loc[ind_1, '1prior'] = df['encoded_message'][ind_1]

In [ ]:
df.to_csv('context_messages_labels_1000_1234.csv')